# Interactive Plotting and Monte Carlo Simulations of CPPI

In [1]:
import ipywidgets as widgets
from IPython.display import display

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import kit as erk
%load_ext autoreload
%autoreload 2

### GBM Simulations

In [2]:
def show_gbm(n_scenarios, mu, sigma):
    """
    Draw the results of a stock price evolution under a Geometric Brownian Motion model
    """
    S0 = 100
    prices = erk.gbm(n_scenarios=n_scenarios, mu=mu, sigma=sigma, S0 = S0 )

    ax = prices.plot(legend=False, color="darkorange", alpha = 0.5, linewidth=2, figsize=(12,5))
    ax.axhline(y = S0, ls=":", color="black")

    # Draw a dot at the origin
    ax.plot(0,S0 , marker='o',color='darkred', alpha=0.2)


In [3]:
gbm_controls = widgets.interactive(show_gbm, 
                                   n_scenarios = widgets.IntSlider(min = 1, max = 100, step=1, value=1), 
                                   mu = (0, 0.2, 0.01),
                                   sigma = (0, 0.3, 0.01)
)

In [4]:
display(gbm_controls)

interactive(children=(IntSlider(value=1, description='n_scenarios', min=1), FloatSlider(value=0.1, description…

### Interactive CPPI Monte Carlo Testing

In [5]:
def show_cppi(n_scenarios = 50, mu = 0.07, sigma = 0.15, m = 3, floor = 0, riskfree_rate = 0.03, y_max = 100, steps_per_year = 12):
    """
    Plot the results of a Monte Carlo Simulation of CPPI

        - n : total scenarios 
        - mu : weighted avg. of portfolio returns 
        - sigma : weighted volatility of portfolio
        - m : Risky asset multiplier
        - floor : maximun acetable proportion of losses by initial investment [S0]
        - R_f : risk free rate (Treasury Bond)
        - y_max : plot y axis scale
        - steps_per_year : CPPI portfolio actualization frequency. ( default: monthly )
    """
    S0  = 100 # Nominal Inicial
    sim_rets = erk.gbm(n_scenarios = n_scenarios, mu = mu, sigma = sigma, prices = False, steps_per_year = steps_per_year)
    risky_r = pd.DataFrame(sim_rets) # Retornos generados con mov. Borowniano




    # Run the back-test

    btr = erk.run_cppi(risky_r = pd.DataFrame(risky_r), riskfree_rate = riskfree_rate, m = m, start = S0 , floor=floor)
    wealth = btr["Wealth"]
    y_max = wealth.values.max()*y_max/100 # y axis Zoom



    # Calculate terminal wealth stats

    terminal_wealth = wealth.iloc[-1]
    
    tw_mean = terminal_wealth.mean()  # ret. promedio
    tw_median = terminal_wealth.median() # ret. medio

    fail = np.less(terminal_wealth, S0*floor)
    n_failures = fail.sum()
    p_fail = n_failures/n_scenarios

    avg_shortfall = np.dot( terminal_wealth - S0 * floor,  fail ) / n_failures if (n_failures > 0) else 0

    

    # Plot

    fig, (wealth_ax, hist_ax) = plt.subplots(nrows = 1, ncols = 2, sharey = True, gridspec_kw={'width_ratios':[3,2]}, figsize=(24, 9))
    plt.subplots_adjust(wspace=0.0) # Adjacent plots
    
    wealth.plot(ax=wealth_ax, legend=False, alpha=0.3, color="darkorange")
    wealth_ax.axhline(y= S0, ls=":", color="black")
    wealth_ax.axhline(y= S0*floor, ls="--", color="limegreen")
    wealth_ax.set_ylim(top=y_max)


    
    terminal_wealth.plot.hist(ax = hist_ax, bins=50, ec='w', fc='darkorange', orientation='horizontal', alpha = 0.8)
    hist_ax.axhline(y= S0, ls=":", color="black")
    hist_ax.axhline(y= tw_mean, ls="--", color="blue")
    hist_ax.axhline(y= tw_median, ls="--", color="indigo")



    hist_ax.annotate(f"Mean: ${int(tw_mean)}", xy=(.6, .9),xycoords='axes fraction', fontsize= 14, color = 'blue')
    hist_ax.annotate(f"Median: ${int(tw_median)}", xy=(.6, .85),xycoords='axes fraction', fontsize= 14, color = 'indigo')
    
    if (floor > 0.01):
        hist_ax.axhline(y=S0*floor, ls="--", color="red", linewidth=3)
        hist_ax.annotate(f'Violations: {n_failures} [{p_fail*100:2.2f}%]\nE(shortfall): ${avg_shortfall:2.2f}', xy=(.6, .7), xycoords='axes fraction', fontsize= 14)



cppi_controls = widgets.interactive(show_cppi, 
                                   n_scenarios=widgets.IntSlider(min=1, max=1000, step=5, value=50), 
                                   mu=(-0.2, 0.2, 0.01),
                                   sigma=(0, 0.30, 0.05),
                                   floor=(0, 2, 0.1),
                                   m=(1, 5, 0.5),
                                   riskfree_rate=(0, 0.05, 0.01),
                                   y_max = widgets.IntSlider(min=0, max=100, step=1, value=100, description="Zoom y axis"),
                                   steps_per_year = widgets.IntSlider(min=1, max=12, step=1, value=12, description="Periods per Year")
                             )
display(cppi_controls)

interactive(children=(IntSlider(value=50, description='n_scenarios', max=1000, min=1, step=5), FloatSlider(val…